In [1]:
import time
from datetime import datetime as dt
from biscoint_api_python import Biscoint
from playsound import playsound
from utils import percent
from configs import logging
from configs import API_KEY,API_SECRET,AMOUNT,MIN_PERCENT_REQUIRED,BASE_URL

In [2]:
bsc = Biscoint(API_KEY, API_SECRET)
endpoints_meta = bsc.get_meta()
rate_limit_offer = endpoints_meta['endpoints']['offer']['post']['rateLimit']
sleep_time_offers = ((rate_limit_offer["windowMs"] / rate_limit_offer["maxRequests"]) / 1000)*2
initial_balance = bsc.get_balance()
last_balance = initial_balance

In [3]:
#requests.get(f"{BASE_URL}/fees").json()
bsc.get_fees()

{'withdrawal': {'BTC': {'rate': '0.0',
   'fixed': {'slow': '0.00051', 'normal': '0.00061', 'fast': '0.00073'}},
  'BRL': {'rate': '0.0',
   'fixed': {'ted': '14.90', 'sameBankTransfer': '14.90'}}}}

In [12]:
buy = bsc.get_offer(op='buy',amount=AMOUNT,isQuote=False)
print(buy)

bsc.confirm_offer(buy['offerId'])

{'offerId': 'A3XbE8T2X9865td78', 'base': 'BTC', 'quote': 'BRL', 'op': 'buy', 'isQuote': False, 'baseAmount': '0.00010000', 'quoteAmount': '32.60', 'efPrice': '326000.00', 'createdAt': '2021-04-19T10:27:42.064Z', 'expiresAt': '2021-04-19T10:27:57.064Z', 'apiKeyId': 'vo3Jt2cEmvPxQZ7NS'}


HTTPError: 403 Client Error: Forbidden for url: https://api.biscoint.io/v1/offer/confirm

In [17]:
while True:
    start_time = dt.now()
    buy = bsc.get_offer(op='buy',amount=AMOUNT,isQuote=False)
    sell = bsc.get_offer(op='sell',amount=AMOUNT,isQuote=False)

    calculated_percent = percent(buy['efPrice'],sell['efPrice'])
    print(f"Percent is {calculated_percent}")

    if(calculated_percent >= MIN_PERCENT_REQUIRED):
        print(f"Arbitrage oportunity: buy:{buy['efPrice']}   sell:{sell['efPrice']}")
        playsound('beep.wav')
        #Execute orders
        executed_sell = bsc.confirm_offer(sell['offerId'])
        executed_buy = bsc.confirm_offer(buy['offerId'])
        last_balance = bsc.get_balance()
        break


    end_time = dt.now()
    seconds_elapsed = (end_time - start_time).total_seconds()

    #print(f"Took {seconds_elapsed} seconds")
    
    time.sleep(sleep_time_offers)

Percent is -0.676
Percent is -0.615
Percent is -0.615
Percent is -0.645
Percent is -0.645
Percent is -0.676
Percent is -0.676
Percent is -0.676
Percent is -0.615


KeyboardInterrupt: 

In [ ]:
#thread_buy = threading.Thread(target=offerFunc,args=('buy',AMOUNT,False,buy))
#thread_sell = threading.Thread(target=offerFunc,args=('sell',AMOUNT,False,sell))
#thread_buy.start()
#thread_sell.start()
#thread_buy.join()
#thread_sell.join()
